<font size = 6>**Pupil Labs Blink Detection**

In [2]:
import pathlib
import numpy as np

from blink_detector.blink_detector import (
    calculate_optical_flow, 
    predict_class_probas, 
    smooth_probas, 
    threshold_probas, 
    compile_into_events, 
    filter_events, 
    extract_blink_events,
)

from blink_detector.helper import (
    preprocess_frames, 
    get_video_frames, 
    get_timestamps,
    OfParams,
    PPParams,
    create_grid,
    get_classifier,
    get_clf_path,
    generate_animation,
)

/cluster/anaconda3/envs/tom_py310/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


<font size = 5>**(1) Run blink detection pipeline using an example Neon recording**

**(1.1) Load example recording and apply preprocessing**

In [3]:
recording_path = pathlib.Path("blink_detector/data/padel_tennis_neon_03-2ded8f56")

left_images, right_images = get_video_frames(recording_path)

timestamps = get_timestamps(recording_path)

# Example recording is Neon.
preprocessed_images = list(preprocess_frames(left_images, right_images, is_neon=True))

left_images = preprocessed_images[0][0]
right_images = preprocessed_images[0][1]

**(1.2) Load classifier as well as default optical flow and postprocessing parameters**

In [7]:
clf = get_classifier(is_neon=True)

of_params = OfParams()
pp_params = PPParams()

grid = create_grid(of_params.img_shape, of_params.grid_size)

**(1.3) Predict blink events from video frames**

In [8]:
images_timestamps = zip(zip(left_images, right_images), timestamps)

x = calculate_optical_flow(images_timestamps, of_params, grid)
x = predict_class_probas(x, clf, of_params)
x = smooth_probas(x, pp_params)
x = threshold_probas(x, pp_params)
x = compile_into_events(x)
x = filter_events(x)

blink_events = list(extract_blink_events(x, pp_params))

<font size = 5><b>Visualize blinks</b><br><br>
<font size=3>In order to visualize the blinks, a white square is drawn into the center of each frame <br>
that is part of a blink event. Next, a short video is generated from the frames.<br>

In [9]:
from IPython.display import HTML
from blink_detector.helper import show_video

# only aniimate the first N samples, otherwise creating the animation can take long
video_length = 3000

blink_indices = np.zeros_like(timestamps)

for blink_event in blink_events:
    on = np.where(blink_event.start_time == timestamps)[0]
    off = np.where(blink_event.end_time == timestamps)[0]
    blink_indices[int(on) : int(off)] = 1

anim = generate_animation(
    left_images[:video_length, :, :],
    right_images[:video_length, :, :],
    indices=blink_indices[:video_length],
)

HTML(anim.to_html5_video())

<font size = 5><b>(2) Blink detection in real time using PL's realtime API</b>

<font size = 3>In the next steps, blink events will be detected in (quasi) real-time using Pupil Lab's Real Time API. <br>
Note that the temporal resolution of the blink detection is restricted by the optical flow parameters<br>
(i.e., the number of layers and the layer interval), as well as postprocessing parameters (e.g. the minimum blink duration).

**(2.1) Load all relevant modules and functions**

In [10]:
from pupil_labs.realtime_api.simple import discover_one_device
from blink_detector.blink_detector import 
from itertools import tee
import time
import nest_asyncio
import cv2

from blink_detector.helper import blink_detection_pipeline, video_stream

**(2.2) Set up real time API**

In [ ]:
# needed when running in notebook
nest_asyncio.apply()

# calling the functions shortly after one another can cause an error
time.sleep(1)

device = discover_one_device()

Make sure that the connected device is indeed the device you would like to connect with. <br>
Printing the device name and its IP should be sufficient to do so.

In [ ]:
print(f"Phone IP address: {device.phone_ip}")
print(f"Phone name: {device.phone_name}")

In [ ]:
# set classifier path
clf_path = get_clf_path(is_neon=True)

In [ ]:
# create three independent generator objects from the video stream
stream_left, stream_right, stream_ts = tee(video_stream(device), 3)

left_images = (left for left, _, _ in stream_left)
right_images = (right for _, right, _ in stream_right)

# timestamps need to be converted to ns
timestamps = (1e9 * timestamp for _, _, timestamp in stream_ts)

for blink_events in blink_detection_pipeline(
    left_images, right_images, timestamps, classifier_path=clf_path
):
    print(blink_events)